In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
import seaborn as sns
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV, SelectFromModel, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
%matplotlib inline

In [ ]:
#Reading the dataset using "read_csv" a prebuild function in the "pandas" package, the "index_col" contains the column to use as the row labels of the DataFrame
Stock = pd.read_csv('AppleDataset.csv',  index_col=0)
df_Stock = Stock
#"rename" prebuild function that enables us to change the name of a column in a Dataframe
df_Stock = df_Stock.rename(columns={'Close(t)':'Close'})
#Display using Head function that returns the dataframe or series with the first few rows (by default 5)
df_Stock.head()

In [ ]:
#Converting columns that contain data type (Dtype) =object to float using the "astype" prebuild function 
#In the same time replacing "$" symbol with an empty space 
df_Stock["Close"] = df_Stock[' Close/Last'].str.replace('$','').astype(float)
df_Stock['Open'] = df_Stock[' Open'].str.replace('$','').astype(float)
df_Stock['High'] = df_Stock[' High'].str.replace('$','').astype(float)
df_Stock['Low'] = df_Stock[' Low'].str.replace('$','').astype(float)

df_Stock['Volume'] = df_Stock[' Volume']
#With the new named columns containing the converted datatypes, we are droping pevious columns
drop = [' Close/Last',' Open',' High',' Low',' Volume']
df_Stock = df_Stock.drop(drop,axis=1)
#Diplaying the new dataset 
df_Stock.head()

In [ ]:
def create_train_test_set(df_Stock):
    #Taking all features as predictors of course droping the close  column...
    """
    the close  column represents the predicted value of close prices in the 
    previous 10 years. so not to make our model overfit and the learning to be biased by these values 
    we drop this column.

    """
    features = df_Stock.drop(columns=['Close'], axis=1)

    #now the target is to predit the values of the close column which represents the predicted 
    #prices by which shares of apple corporation will be sold (or bought).
    target = df_Stock['Close']
    

    data_len = df_Stock.shape[0]
    print('Historical Stock Data length is - ', str(data_len))

    #create a chronological split for train and testing
    train_split = int(data_len * 0.7)
    print('Training Set length - ', str(train_split))

    val_split = train_split + int(data_len * 0.2)
    print('Validation Set length - ', str(int(data_len * 0.2)))

    print('Test Set length - ', str(int(data_len * 0.1)))

    # Splitting features and target into train, validation and test samples 
    X_train, X_val, X_test = features[:train_split], features[train_split:val_split], features[val_split:]
    Y_train, Y_val, Y_test = target[:train_split], target[train_split:val_split], target[val_split:]

    #print shape of samples
    print(X_train.shape, X_val.shape, X_test.shape)
    print(Y_train.shape, Y_val.shape, Y_test.shape)
    
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [ ]:
X_train, X_val, X_test, Y_train, Y_val, Y_test = create_train_test_set(df_Stock)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd

# Load stock price data
stock_prices = df_Stock

# Define window size and step size
window_size = 30
step_size = 7

# Initialize model
lr = LinearRegression()

# Iterate over time periods
mse_scores = []
for i in range(window_size, len(stock_prices), step_size):
    # Split data into training and test sets
    X_train = stock_prices.iloc[i-window_size:i, 1:].values
    y_train = stock_prices.iloc[i-window_size:i, 0].values
    X_test = stock_prices.iloc[i:i+step_size, 1:].values
    y_test = stock_prices.iloc[i:i+step_size, 0].values

    # Fit model on training data
    lr.fit(X_train, y_train)

    # Make predictions on test data
    y_pred = lr.predict(X_test)

    # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate average mean squared error
avg_mse = sum(mse_scores) / len(mse_scores)
print("Average MSE: ", avg_mse)

In [ ]:
from sklearn.linear_model import LinearRegression #uploading the linear regression model 

lr = LinearRegression()
#fit() is implemented by every estimator and it accepts an input for the sample data ( X_train ) and for supervised models it also accepts an argument for labels (Y_train)
lr.fit(X_train, Y_train)

In [ ]:
#Displaying the models coefficients and intercept 
print('LR Coefficients: \n', lr.coef_)
print('LR Intercept: \n', lr.intercept_)

In [ ]:
print("Performance (R^2): ", lr.score(X_train, Y_train))

In [ ]:
#Calculating the regression error in order to be used while evaluating the model's performance
def get_mape(y_true, y_pred): 
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

5.1. Training the model 

In [ ]:
#In this case we are using linear regression 

Y_train_pred = lr.predict(X_train)
Y_val_pred = lr.predict(X_val)
Y_test_pred = lr.predict(X_test)

In [ ]:
#Using different metrics (prebuild function of sklearn) to evaluate the model's performance
print("let's get metrics for training ...........\n\n")

print("Training R-squared: ",round(metrics.r2_score(Y_train,Y_train_pred),2))
print("Training Explained Variation: ",round(metrics.explained_variance_score(Y_train,Y_train_pred),2))
print('Training MAPE:', round(get_mape(Y_train,Y_train_pred), 2)) 
print('Training Mean Squared Error:', round(metrics.mean_squared_error(Y_train,Y_train_pred), 2)) 
print("Training RMSE: ",round(np.sqrt(metrics.mean_squared_error(Y_train,Y_train_pred)),2))
print("Training MAE: ",round(metrics.mean_absolute_error(Y_train,Y_train_pred),2))

print("\n\nNow it is time for validation ...........\n\n")

print("Validation R-squared: ",round(metrics.r2_score(Y_val,Y_val_pred),2))
print("Validation Explained Variation: ",round(metrics.explained_variance_score(Y_val,Y_val_pred),2))
print('Validation MAPE:', round(get_mape(Y_val,Y_val_pred), 2)) 
print('Validation Mean Squared Error:', round(metrics.mean_squared_error(Y_train,Y_train_pred), 2)) 
print("Validation RMSE: ",round(np.sqrt(metrics.mean_squared_error(Y_val,Y_val_pred)),2))
print("Validation MAE: ",round(metrics.mean_absolute_error(Y_val,Y_val_pred),2))


print("\n\nlet's go directly and dicover how our model did for the testing dataset ...\n\n")


print("Test R-squared: ",round(metrics.r2_score(Y_test,Y_test_pred),2))
print("Test Explained Variation: ",round(metrics.explained_variance_score(Y_test,Y_test_pred),2))
print('Test MAPE:', round(get_mape(Y_test,Y_test_pred), 2)) 
print('Test Mean Squared Error:', round(metrics.mean_squared_error(Y_test,Y_test_pred), 2)) 
print("Test RMSE: ",round(np.sqrt(metrics.mean_squared_error(Y_test,Y_test_pred)),2))
print("Test MAE: ",round(metrics.mean_absolute_error(Y_test,Y_test_pred),2))

print("\nIt seems WE DID WELL ")

In [ ]:
#Displaying the values of actual price at a closing trading day and the predicted value of the price 
df_pred = pd.DataFrame(Y_val.values, columns=['close Actual'], index=Y_val.index)
df_pred['Close Predicted'] = Y_val_pred
df_pred = df_pred.reset_index()
df_pred
